In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
import numpy as np
import os
import time
import json
import pickle
from glob import glob
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [ ]:
import keras as K

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)

In [ ]:
annotation_folder = '../Dataset/MSCOCO/annotations/'
image_folder = '../Dataset/MSCOCO/train2014/'

In [ ]:
annotation_file = annotation_folder + 'captions_train2014.json'

# Read the json file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [ ]:
NUM_SAMPLES = 60

# Store captions and image names
all_captions = []
all_img_paths = []

for annot in annotations['annotations']:
    caption = annot['caption']
    image_id = annot['image_id']
    img_path = image_folder + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_paths.append(img_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
all_captions, all_img_paths = shuffle(all_captions, all_img_paths, random_state=1)
train_captions = all_captions[:NUM_SAMPLES]
train_img_paths = all_img_paths[:NUM_SAMPLES]

In [ ]:
print("len train_captions :", len(train_img_paths))
print("len all_captions :", len(all_img_paths))

## Encoder

In [ ]:
MODEL_TYPE = "xception"


def get_encoder(model_type=MODEL_TYPE):

    if model_type == "xception":
        cnn_preprocessor = tf.keras.applications.xception
        cnn_model = tf.keras.applications.Xception(include_top=False, weights='imagenet')

    elif model_type == "inception_v3":
        cnn_preprocessor = tf.keras.applications.inception_v3
        cnn_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

    input_layer = cnn_model.input
    output_layer = cnn_model.layers[-1].output # use last hidden layer as output
    
    encoder = tf.keras.Model(input_layer, output_layer)
    encoder_preprocessor = cnn_preprocessor
    
    return encoder, encoder_preprocessor


encoder, encoder_preprocessor = get_encoder(MODEL_TYPE)

In [ ]:
IMAGE_SIZE = (299, 299)


def load_image(image_path):

    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = encoder_preprocessor.preprocess_input(image)
    
    return image, image_path

## Prepare Image dataset

In [ ]:
BATCH_SIZE = 8

# Get unique images
unique_train_img_paths = sorted(set(train_img_paths))

# Prepare dataset
image_dataset = tf.data.Dataset.from_tensor_slices(unique_train_img_paths)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) # use max num of CPU
image_dataset = image_dataset.batch(BATCH_SIZE)

In [ ]:
estimated_batch_count = NUM_SAMPLES / BATCH_SIZE + 1
print("estimated_batch_count", estimated_batch_count)

In [ ]:
# Preprocessed image (batch)
for batch_imgs, batch_img_paths in tqdm(image_dataset):
    
    # get context vector of batch images
    batch_features = encoder(batch_imgs)
    
    # flatten 2D cnn result into 1D for RNN decoder input
    # (batch_size, 10, 10, 2048)  => (batch_size, 100, 2048)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))
    
    # Cache preprocessed image
    for image_feature, image_path in zip(batch_features, batch_img_paths):
        image_path = image_path.numpy().decode("utf-8")
        np.save(image_path, image_feature.numpy())

## Prepare Tokenizer

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer

In [ ]:
TOKENIZER = "BERT"
VOCAB_SIZE = 5000  # Choose the top-n words from the vocabulary

In [ ]:
class BertTokenizerWrapper(BertTokenizer):

    def texts_to_sequences(self, texts):
        """
        convert batch texts into indexed version
        eg: ['an apple', 'two person']
        output: [[1037,17260], [2083, 2711]] 
        """
        
        tokenized_texts = [self.tokenize(x) for x in texts]
        token_ids = [self.convert_tokens_to_ids(x) for x in tokenized_texts]
        
        return token_ids

In [ ]:
if TOKENIZER == "BERT" :

    # Load pre-trained BERT tokenizer (vocabulary)
    tokenizer = BertTokenizerWrapper.from_pretrained('bert-base-uncased')
    
else : 
    
    # use default keras tokenizer
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="[UNK]")
    tokenizer.fit_on_texts(train_captions)    
    tokenizer.word_index['[PAD]'] = 0
    tokenizer.index_word[0] = '[PAD]'

In [ ]:
train_captions = tokenizer.texts_to_sequences(train_captions)

## Pad sequence

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# If you do not provide a max_length value, pad_sequences calculates it automatically
MAX_LENGTH = None  # use <int> or None

In [ ]:
train_captions = pad_sequences(train_captions, maxlen=MAX_LENGTH, padding='post', truncating="post")

## Create dataset object

In [ ]:
# Load the numpy files
def load_image_npy(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8') + '.npy')
    return img_tensor, cap

In [ ]:
# Create dataset object
from tensorflow.data import Dataset
dataset = Dataset.from_tensor_slices((train_img_paths, train_captions))

# Use map to load the numpy files in parallel
# wrap function into numpy function
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          load_image_npy, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# Shuffle and batch
dataset = dataset.shuffle(1000).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Split train eval test

In [ ]:
# Split dataset 

TRAIN_SPLIT = 0.7
EVAL_SPLIT = 0.15
TEST_SPLIT = 0.15  # approx

n_batch = int(NUM_SAMPLES / BATCH_SIZE) + 1
n_train = int(n_batch * 0.7)
n_eval = int(n_batch * 0.15)
n_test = n_batch - (n_train + n_eval)

train_dataset = dataset.take(n_train)
eval_dataset = dataset.skip(n_train).take(n_eval)
test_dataset = dataset.skip(n_train + n_eval)


"""
dataset => tuple of (image, captions)
image   => (batch_size = 8, 100, 2048)
caption => (batch_size = 8, max_length)
"""

In [ ]:
print("train: {} batches, (total : {})".format(n_train, n_train * BATCH_SIZE))
print("eval : {} batches, (total : {})".format(n_eval, n_eval * BATCH_SIZE))
print("test : {} batches, (total : {} (aprx))".format(n_test, n_test * BATCH_SIZE))

# Model

## Encoder

In [ ]:
class CNN_Encoder(tf.keras.Model):
    
    # Image features are extracted and saved already
    # This encoder passes those features through a Fully connected layer


    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.fc = tf.keras.layers.Dense(embedding_dim, activation="relu")

    def call(self, x):
        
        x = self.fc(x)
        # shape after fc == (batch_size, 64, embedding_dim)

        return x

In [ ]:
class BahdanauAttention(tf.keras.Model):
    
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)
        # hidden shape == (batch_size, hidden_size)

        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, 64, hidden_size)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        # attention_weights shape == (batch_size, 64, 1)
        # you get 1 at the last axis because you are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class RNN_Decoder(tf.keras.Model):
    
    def __init__(self, embedding_dim, units, vocab_size):
        
        super(RNN_Decoder, self).__init__()
        self.units = units

        
        ## TODO: add BERT here
        ## Should have interoperability with BERT
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert.eval();
        
        
        
        ## TODO: allow choose lstm vs gru
        self.lstm = tf.keras.layers.LSTM(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        
        
        
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size) ## same size as vocab to choose words
        
        """
        if use bert, vocab size = len(tokenizer.vocab.keys())

        """

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        """ this using inject method,
        image's feature injected at every timestep """
        
        
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        
        ## TODO: inject image here ?
        ## TODO: add PPLM framework here
        ## feed-forward, find gradient, apply gradient, feed-forward again 
        
        
        ## Concat context vector + hidden state => inject method
        ## TODO : use merge method, pre method
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        ## TODO : allow choose gru vs lstm
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        output, last_memory_state, last_carry_state = self.lstm(x)

        
        ## TODO: add another attention layer ? 

        
        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        
        ## TODO : merge method, concat context vector + hiddens state here .
        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

## !!! Check

check real vs pred dimension

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)



"""
Detail info:

real => (batch_size = 8, (1))
pred => (batch_size = 8, 5001)
mask => Tensor("Cast:0", shape=(batch_size,), dtype=float32)
loss_ => Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(batch_size,), dtype=float32)

return => Tensor("Mean:0", shape=(), dtype=float32)

"""

In [ ]:
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    ## TODO: allow using diffrent strategy for inititalizer ? 
    dec_input = tf.expand_dims([tokenizer.word_index['[START]']] * target.shape[0], 1)

    
    with tf.GradientTape() as tape:
        
        ## Get image context vector
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            # loss => Tensor("add:0", shape=(), dtype=float32)
            loss += loss_function(target[:, i], predictions)
            
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
@tf.function
def eval_step(img_tensor, target):
    """
    basically same as train_step, but doesn't apply gradient 
    """
    
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    ## TODO: allow using diffrent strategy for inititalizer ? 
    dec_input = tf.expand_dims([tokenizer.word_index['[START]']] * target.shape[0], 1)
    
    
    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)
            
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))
    
    return loss, total_loss

In [ ]:
## Customized to enable multiple images, and incorporate PPLM framework


def custom_evaluate(image, supporting_images):
    
    attention_plot = np.zeros((max_length, attention_features_shape))

    # initializing the hidden state for decoder
    hidden = decoder.reset_state(batch_size=1)

    # Extract image features
    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)
    
    
    ## TODO: extract features from supporting images
    ## TODO: average those features into single context vector
    
    
    ## TODO: allow using diffrent strategy for inititalizer ? 
    ## TODO: BERT doesn't have word_index
    dec_input = tf.expand_dims([tokenizer.word_index['[START]']], 0)
    result = []

    for i in range(max_length):
        """
        Currently using inject method 
        (image feature injected at every timestep)
        
        TODO: test merge approach & pre, par inject
        """
        
        ## image feature inject's method
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        
        
        ## TODO : apply PPLM here
        ## check loss (prediction - context vector of supporting images)
        ## apply gradient : hidden_state += diffrence(pred, supporting img vectors) (after n-iteration)
        ## re compute predictions
        
        
        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        
        
        ## TODO: Allow using BERT tokenizer
        ## TODO: Revert predictions to index and words
        result.append(tokenizer.index_word[predicted_id])   # <<< return back id to text

        
        if tokenizer.index_word[predicted_id] == '[END]':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
embedding_dim = 768
units = 200


encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, VOCAB_SIZE)

## Setup training

In [ ]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

## Actual training

In [ ]:
EPOCHS = 7

for epoch in range(start_epoch, EPOCHS):
    
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in tqdm(enumerate(train_dataset)):
        
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
            
        
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1, total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))